In [34]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        # 매우 큰 데이터 구조체를 표현하는 문자열을 축약해서 생성한다. (30자 제한) 
        return "Sentence(%s)" % reprlib.repr(self.text)    

In [2]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [3]:
# Sentence 객체는 반복할 수 있다.
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [4]:
# 반복할 수 있으므로 Sentence 객체는 리스트 혹은 다른 반복형을 생성하기 위한 입력으로 사용할 수 있다. 
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

---

In [9]:
RE_WORD = re.compile('\w+')
RE_WORD.findall("The King of Passion123 asd 214@#!#124ipjdlkasmk")

['The', 'King', 'of', 'Passion123', 'asd', '214', '124ipjdlkasmk']

In [10]:
class Foo:
    def __iter__(self):
        pass

In [36]:
from collections import abc
print(issubclass(Sentence, abc.Iterable))
print(isinstance(Sentence, abc.Iterable))

False
False


In [12]:
f = Foo()
isinstance(f, abc.Iterable)

True

## 14.2 반복형과 반복자

In [13]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [14]:
s = 'ABC'
it = iter(s)

while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


In [19]:
from collections import Iterable
from abc import abstractmethod 

class Iterator(Iterable):
    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        raise StopIteration
        
    def __iter__(self):
        return self
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any("__next__" in B.__dict__ for B in C.__mro__) and
               any("__iter__" in B.__dict__ for B in C.__mro__)):
                return True
                
        return NotImplemented

In [20]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)   # 반복자를 가져온다. 
it

In [21]:
next(it)

'Pig'

In [22]:
next(it)

'and'

In [23]:
next(it)

'Pepper'

In [26]:
next(it)   # 더 이상 단어가 없으므로 예외를 발생시킨다. 

StopIteration: 

In [24]:
list(it)  # 소진된 후에는 반복자가 필요 없다. 

[]

In [25]:
list(iter(s3))   # 다시 반복하려면 생성자를 새로 만들어야 한다. 

['Pig', 'and', 'Pepper']

## 14.3 고전적인 반복자

In [37]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)    
    
    def __iter__(self):    # 반복자 객체를 생성해서 반환함으로써 반복형 프로토콜 구현
        return SentenceIterator(self.words) 
    
class SentenceIterator:
    def __init__(self, words):
        self.words = words    # 단어 리스트에 대한 참조를 담고 있다. 
        self.index = 0    # 다음에 가져올 단어를 결정하기 위해 self.index를 사용한다. 
        
    def __next__(self):
        try:
            word = self.words[self.index]    # self.index 에 있는 단어를 가져온다, 
        except IndexError:
            raise StopIteration()    # self.index 에 단어가 없으면 예외 발생
        self.index += 1    # 인덱스 증가 
        return word    # 단어 반환
    
    def __iter__(self):
        return self    

In [40]:
print(hasattr(Iterator, '__iter__'))
print(hasattr(Iterator, '__next__'))

True
True


In [31]:
s = Sentence('"The time has come," the Walrus said,')
# Sentence 객체는 반복할 수 있다.
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [32]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

## 14.4 Sentence 버전 #3 : 제너레이터 함수

In [41]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
            
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)    
    
    def __iter__(self):
        for word in self.words:
            yield word
        return 

In [42]:
def gen_123():
    yield 1
    yield 2
    yield 3

In [44]:
gen_123()

<generator object gen_123 at 0x1077c00b0>

In [45]:
for i in gen_123():
    print(i)

1
2
3


In [46]:
g = gen_123()
next(g)

1

In [47]:
next(g)

2

In [48]:
next(g)

3

In [49]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')
    
for c in gen_AB():
    print('-->', c)

start
--> A
continue
--> B
end.


## 14.6 Sentence 버전 #5 : 제너레이터 표현식

In [50]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')
    
res = [x*3 for x in gen_AB()]

start
continue
end.


In [51]:
for i in res:
    print('-->', i)

--> AAA
--> BBB


In [52]:
res2 = (x*3 for x in gen_AB())
res2

<generator object <genexpr> at 0x1077b4660>

In [53]:
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


### 14.8.1 itertools를 이용한 등차수열

In [60]:
import itertools
gen = itertools.count(1, .5)
next(gen)

1

In [61]:
next(gen)

1.5

In [62]:
next(gen)

2.0

In [63]:
next(gen)

2.5

In [65]:
gen = itertools.takewhile(lambda n:n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [66]:
def vowel(c):
    return c.lower() in 'aeiou'

list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [67]:
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [68]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [69]:
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

---

In [70]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
import itertools
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [71]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [72]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [73]:
import operator
list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [74]:
list(itertools.accumulate(range(1, 11), operator.mul))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

---

In [75]:
list(enumerate('albatroz', 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [76]:
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [77]:
list(map(operator.mul, range(11), [2, 4, 8]))

[0, 4, 16]

In [78]:
list(map(lambda a, b : (a, b), range(11), [2, 4, 8]))

[(0, 2), (1, 4), (2, 8)]

In [79]:
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

---

In [80]:
list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [82]:
list(itertools.chain(enumerate('ABC')))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [83]:
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [84]:
list(zip('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [85]:
list(zip('ABC', range(5), [10, 20, 30, 40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [87]:
list(itertools.zip_longest('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [88]:
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

---

In [89]:
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [90]:
suits = 'spades hearts diamonds clubs'.split()
list(itertools.product('AK', suits))

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [91]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [92]:
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [93]:
rows = itertools.product('AB', range(2), repeat=2)
for row in rows: print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


---

In [94]:
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [95]:
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [96]:
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [97]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

---

In [98]:
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat', 
          'dolphin', 'shark', 'lion']

animals.sort(key=len)

In [99]:
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

In [100]:
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']


---

In [103]:
s = 'ABC' 
t = tuple(range(3))

def chain(*iterables):
    for i in iterables:
        yield from i
        
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

---

In [104]:
all([1, 2, 3])

True

In [105]:
all([1, 0, 3])

False